[py080 Простая программа работы с СОМ портом на Python](#py080-Простая-программа-работы-с-СОМ-портом-на-Python)   
[py081 Использование TFTP на Python 3](#py081-Использование-TFTP-на-Python-3)   
[py082 Формирование и отправка DNS запроса на Python 3](#py082-Формирование-и-отправка-DNS-запроса-на-Python-3)  
[py083 Парсим простые сайты на Python3](#py083-Парсим-простые-сайты-на-Python3)  
[py084 Обмен данными по ТСР](#py084-Обмен-данными-по-ТСР)  
[py085 Парсим страницы сайта с отправкой формы](#py085-Парсим-страницы-сайта-с-отправкой-формы)  
[py086 Сетевая игра часть 1 интерфейс клиента](#py086-Сетевая-игра-часть-1-интерфейс-клиента)  
[py087 Сетевая игра часть 2 подключаем ТСР](#py087-Сетевая-игра-часть-2-подключаем-ТСР)  
[py088 Сетевая игра часть 3 подключаем Эхо-сервер](#py088-Сетевая-игра-часть-3-подключаем-Эхо-сервер)

#### py080 Простая программа работы с СОМ портом на Python

In [6]:
#-- coding:utf8

#-- библиотеки
import time
import serial
import threading
from time import *

from tkinter import * 
from tkinter import ttk
from tkinter.font import Font
from tkinter import filedialog as tkFileDialog

#-- настройки оформления
root = Tk()
#-- стили
stl = ttk.Style()
dFont = ('Helvetica',12,'bold')
stl.configure('.', font=dFont, background="white",foreground="black")
stl.configure('TLabel',foreground='black',sticky=E,padx=10)
stl.configure('TEntry',padx=5,pady=5,sticky=W,font=dFont)
stl.configure('TButton',padx=5,pady=5,sticky=W,font=dFont)
stl.configure('TComboBox',padx=5,pady=5,sticky=W,width=10,font=dFont)

##-- упрощенные классы виджетов
###-- кнопка с набором параметров

class Buttongrid(ttk.Button):
    def __init__( self, panel, row_num, col_num, btn_text, btn_width = 10, btn_fnc = ""):
        ttk.Button.__init__(self, panel, text = btn_text, width = btn_width)
        self.grid(row = row_num, column = col_num, sticky=E+N, pady = 5 , padx = 5)
        if btn_fnc != "":
            self.bind("<Button-1>", btn_fnc)

###-- поле ввода с меткой
class Labelentry(ttk.Entry):
    def __init__( self, panel, row_num, col_num, lab_text = "",
                  ent_width = 10,  init_val = "",
                  fnc_return = "", col_span = 2 ):
        self.var = StringVar()
        ttk.Entry.__init__( self, panel,  width = ent_width, textvariable = self.var, font = dFont)
        self.var.set(init_val)
        if  fnc_return  !=  "":
            self.bind('<Return>', fnc_return)

        if len(lab_text) > 0:
            self.lab = ttk.Label( panel, text=lab_text , font = dFont )
            self.lab.grid( row = row_num, column = col_num, sticky = E , padx = 5)
        self.grid(row = row_num, column = col_num + 1, columnspan = col_span, sticky=W, pady = 5, padx=5 )

    def  get(self):
        return  self.var.get()

    def  set( self, new_text ):
        self.var.set(new_text)

###-- поле чекбокса с меткой
class Labelcheckbutton(ttk.Checkbutton):
    def __init__(self,panel,row_num,col_num,lab_text="",ent_width=0,init_val=0, fnc_sel = 0):
        self.var =IntVar()
        self.var.set(init_val)
        if ent_width==0:
            ent_width=len(lab_text)
        ttk.Checkbutton.__init__(self,panel,width=ent_width,variable=self.var,
                                text=lab_text,command=fnc_sel)
        self.var.set(init_val)
        self.grid(row=row_num,column=col_num,sticky=E,pady=5,padx=5)
    def get(self):
        return self.var.get()
    def  set( self, new_text ):
        self.var.set(new_text)

###-- раскрывающийся список с меткой
class Labelcombobox(ttk.Combobox):
    def __init__( self, panel, row_num, col_num, lab_text = "",  cbx_width = 20, cbx_height = 6, lst_values = [1,2,3], fnc_sel = 0, row_span = 1, col_span = 1):
        if len(lab_text) > 0:
            self.lab = ttk.Label( panel, text=lab_text , font = dFont )
            self.lab.grid( row = row_num, column = col_num, sticky = E , padx = 5, columnspan = col_span)

        ttk.Combobox.__init__( self, panel, values = lst_values, height=cbx_height, width = cbx_width, font = dFont)
        self.set(lst_values[0])   
        self.grid(row = row_num, column = col_num + 1, sticky = W, pady = 5, padx = 5)
        if fnc_sel != 0:
            self.bind("<<ComboboxSelected>>", fnc_sel)

    def  get_index(self):
        try:
            return  self.current()
        except:
            return -1

    def  set_index(self, index):
        try:
            self.current( index )
            return index 
        except:
            return  -1

    def load(self, file_name):
        if len(file_name) >=1:
            self["values"]=[]
            fhn = open(file_name )
            self["values"] = fhn.readlines()
            fhn.close()
            self.set(self["values"][0])
         
###-- список с меткой
class Labellistbox(Listbox):
    def __init__( self, panel, row_num, col_num, lab_text = "",  lbx_width = 20, lbx_height = 6, lst_values = [1,2,3], fnc_dbl = 0, row_span = 1, col_span = 2):
        self.panel = ttk.Frame(panel )

        Listbox.__init__( self, self.panel, width = lbx_width, height=lbx_height, font = dFont)

        if len(lab_text) > 0:
            self.lab = ttk.Label( panel, text=lab_text , font = dFont )
            self.lab.grid( row = row_num, column = col_num, sticky = W , padx = 5, columnspan = col_span)
        self.panel.grid(row = row_num+1, column = col_num, sticky = W , padx = 5, columnspan = col_span, pady = 5)

        for str_lbx in lst_values:
            Listbox.insert(self, END, str_lbx)

        self.pack(side="left", fill="y")
        self.scbr = Scrollbar(self.panel, orient="vertical")
        self.scbr.pack(side="right", fill="y")

        self.scbr.config(command=self.yview)
        self.config(yscrollcommand=self.scbr.set)

        if fnc_dbl != 0:
            self.bind('<Double-Button-1>', fnc_dbl)

    def  append( self, new_text ):
        Listbox.insert(self, END, new_text )
        
            
    def  get(self):
        try:
            return Listbox.get(self, self.curselection()[0])
        except:
            return ""

    def  get_index(self):
        try:
            return  self.curselection()[0]
        except:
            return -1

    def  set_index(self, index):
        try:
            #self.selection_clear()
            self.select_clear(0, "end")
            self.selection_set( index )
            self.see(index)
            self.activate(index)
            self.selection_anchor(index)
            return index 
        except:
            return  -1

    def  clear(self):
        Listbox.delete(self, 0, END)
    
    def  insert( self, new_text ):
        sel_num = self.get_index()
        if sel_num >= 0:
            Listbox.insert(self, sel_num, new_text )
        else:
            Listbox.insert(self, END, new_text )
            
    def  delete(self):
        sel_num = self.get_index()
        if sel_num >= 0:
            Listbox.delete( self, sel_num )

    def save(self, file_name):
        if len(file_name) >=1:
            fhn = open(file_name, "w")
            lst = list(Listbox.get(self, 0, END))
            lst.append("")
            fhn.write('\n'.join(lst))
            fhn.close()
            
    def load(self, file_name):
        if len(file_name) >=1:
            fhn = open(file_name )
            lst_values = fhn.readlines()
            fhn.close()

            Listbox.delete(self, 0, END)
            for str_loc in lst_values:
                Listbox.insert(self, END, str_loc.strip())
    
# ttk.Label(root, text = '' ).grid(row = 0, column = 0, columnspan = 4) 

#-- панель с заголовком
pnl_head = ttk.Frame(root,height=100)
pnl_head.pack(side='top',fill='x')
##-- заголовко программы
ttk.Label(pnl_head,text='Терминал версия 1.00 ').pack()
ttk.Label(pnl_head,text='').pack()

#-- панель для содержимого
pnl_main=ttk.Frame(root,height=800)
pnl_main.pack(side='bottom',fill='both',expand=1)
##-- левая панель
pnl_left=ttk.Frame(pnl_main,width=800)
pnl_left.pack(side='left',fill='both',expand=1)

###-- листбокс отправленных строк
lst_log=['log']
lst_width=60
def fnc_lbxlogdbl(ev):
    edt_strout.set(lbx_log.get()[2:])
    
lbx_log =Labellistbox(pnl_left,0,0,u"журнал связи:",lst_width)
###-- поле ввода отправляемой строки
row_num=16
def fnc_sendstrout(ev):
    global ser
    global add_0D
    if type(ser)==str:     #-- отработать ошибку:
        lbx_log.append(" ошибка: включите  COM порт")
        return
    loc_str=edit_strout.get()    #-- записать строку отпрвки
    str_out=loc_str.split(';')[0].strip() #-- отрезать комментарий)
    str_out=str_out.encode('utf8')
    
    if len(str_out) < 1:     #-- отработать комментарий
        lbx_log.append(" ",+loc_str)
        return
    
    lbx_log.append("<="+loc_str) #--  подготовить строку
    if add_0D:
        str_out+="\r"
    str_out+="\n"
    ser.write(str_out)
                                                     
edt_strout=Labelentry(pnl_left,row_num,0, u'строка:',lst_width-15,'AT',fnc_sendstrout)
###-- кнопка очистки отправляемой строки
row_num+=1
def fnc_clearstrout(event):
    edt_strout.set('')
btn_clearstrout=Buttongrid(pnl_left,row_num,1,u'очистить',10,fnc_clearstrout)
###-- кнопка отправки введенной строки
btn_sendstrout=Buttongrid(pnl_left,row_num,3,u'отправить',10,fnc_sendstrout)
###-- листбокс запомненных значений (программа)
row_num+=1
hgt_frmlbx=8

lst_prog=['AT','ATE0']
def fnc_lbxdbl(ev):
    edt_strout.set(lbx_prog.get())
    fnc_sendstrout(ev)
    
lbx_prog=Labellistbox(pnl_left,row_num,0,u"список строк:",
                lst_width,8,lst_prog,fnc_lbxdbl,hgt_frmlbx,4)
    
###-- кнопка очистки всех строк листбокса программы
row_num=row_num+1+hgt_frmlbx
def fnc_clearlstout(event):
    lbx_prog.clear()
btn_clearlbxout=Buttongrid(pnl_left,row_num,0,u'очистить',10,fnc_clearstrout)
###-- кнопка удаления строки из листбокса программы
def fnc_dellstout(event):
    lbx_prog.delete()
btn_dellstout=Buttongrid(pnl_left,row_num,1,u'удалить',10,fnc_dellstout)
###-- кнопка вставки строки в листбокс программы
def fnc_inslbxout(event):
    lbx_prog.insert(edt_strout.get())
btn_inslbxout=Buttongrid(pnl_left,row_num,2,u'вставить',10,fnc_inslbxout)

###-- кнопка добавления строки в листбокс программы
def fnc_addlbxout(event):
    lbx_prog.append(edt_strout.get())
btn_addlbxout=Buttongrid(pnl_left,row_num,3,u'добавить',10,fnc_addlbxout)
###-- кнопка записи программы в файл
def fnc_save(event):
    file_name = tkFileDialog.SaveAs(root, filetypes = [('*.txt files', '.txt')]).show()
    if file_name != '':
        if file_name[-4:]!=".txt":
            file_name += ".txt"
        lbx_prog.save(file_name)

btn_save = Buttongrid( pnl_left, row_num, 2, u'записать', 10, fnc_save)

###-- кнопка чтения программы из файла
def fnc_load(event):
    file_name = tkFileDialog.Open(root, filetypes = [('*.txt files', '.txt')]).show()
    if file_name != '':
        if file_name[-4:]!=".txt":
            file_name += ".txt"
        lbx_prog.save(file_name)

btn_load = Buttongrid( pnl_left, row_num, 3, u'прочитать', 10, fnc_load)

##-- правая панель с элементами управления
pnl_ctrl=ttk.Frame(pnl_main,width=100)
pnl_ctrl.pack(side='right',fill='y',padx=20)
row_num=0

###-- глобальные переменные связи
com_port = "COM1"
com_speed = 9600
com_parity = "N"
com_stopbits = 1
com_bytesize = 8
ser = ""  #-- COM порт для связи     
add_0D=0  #--  признак автодобавления 0D в конце строки 
###-- заголовок правой панели
ttk.Label(pnl_ctrl,text='параметры соединения').grid(row=row_num,column=0)
###-- найти возможные COM порты
lst_ports=[]
ports=['COM%s'%(i+1)for i in range(20)]
for port in ports:
    try:
        s=serial.Serial(port)
        s.close()
        lst_ports.append(port)
    except(OSError,serial.SerialException):
        pass
com_port=lst_ports[0]

###-- комбобокс для выбора COM порта
def fn_cbxport(ev): # получить значение из комбобокса порта
    global com_port
    com_port=cbx_port.get()
    fn_portoff(ev)
cbx_port=Labelcombobox(pnl_ctrl,row_num,0,'порт:',10,1,lst_ports,fn_cbxport)
###-- список значений скорости и комбобокс для ее выбора
row_num+=1
lst_comspeed=['1200','2400','4800','9600','19200','28800','38400','57600',
              '115200']
def fn_cbxspeed(ev): # получить значение из комбобокса скорости
    global com_speed
    com_speed=int(cbx_speed.get())
    fn_portoff(ev)
cbx_speed = Labelcombobox(pnl_ctrl,row_num,0,'скорость:',10,1,lst_comspeed, fn_cbxspeed)
cbx_speed.set(com_speed)
###-- комбобокс для выбора длины слова
row_num+=1
def fn_cbxwordlen(ev):   # получить значение из комбобокса длины слова
    global com_bytesize
    com_bytesize=int(cbx_wordlen.get())
    fn_portoff(ev)
cbx_wordlen = Labelcombobox(pnl_ctrl,row_num,0,'длина слова:',10,1,[5,7,8], fn_cbxwordlen)
cbx_wordlen.set(str(com_bytesize))
###-- комбобокс для выбора проверки на четность
row_num+=1
def fn_cbxparity(ev):   # получить значение из комбобокса четности
    global com_parity
    parity=cbx_parity.get()
    dict_parity={u"нет":"N",u"четно":"E",u"нечет":"O"}
    com_parity = dict_parity[parity]
    fn_portoff(ev)
    
cbx_parity = Labelcombobox(pnl_ctrl,row_num,0,'четность:',10,1,["нет","четно","нечетно"], fn_cbxparity)
cbx_parity.set("нет")

###-- комбобокс для выбора для выбора стоповых битов
row_num+=1
def fn_cbxstopbits(ev):   # получить значение из комбобокса стоповых битов
    global com_stopbits
    com_stopbits=int(cbx_stopbits.get())
    fn_portoff(ev)
    
cbx_stopbits = Labelcombobox(pnl_ctrl,row_num,0,'стоп битов:',10,1,["1","2"], fn_cbxstopbits)
cbx_stopbits.set("1")

###-- чекбокс добавки 0x0D
row_num+=1
def fn_add0D(ev):   # установить/сбосить режим добавления 0D в конце посылки
    global add_0D
    add_0D=cbt_add0D.get()
    
cbt_add0D = Labelcheckbutton(pnl_ctrl,row_num,0,'добавить 0D',12,0, fn_add0D)
add_0D=0
cbt_add0D.set(add_0D)


###-- строка активности COM порта
row_num+=1
edt_mode=Labelentry(pnl_ctrl,row_num,0,u'COM порт:',12,'отключен')
###-- кнопка отключить COM порт
row_num+=1
link_mode=0
def fn_portoff(ev):
    global link_mode
    global ser
    
    if link_mode==1:
        try:
            ser.close()
            link_mode=0
            edt_mode.set("отключен")
            
        except(OSError, serial.SerialException):
            edt_mode.set("ошибка")
    btn_portoff=Buttongrid(pnl_ctrl, row_num,0,u'отключить',10,fn_portoff)
    
    def fn_porton(ev):
        global link_mode
        global ser
        
        if link_mode == 0:
            try:
                ser.close()
            except:
                pass
            
            try:
                ser=serial.Serial(
                    port=com_port,baudrate=com_speed,parity=com_parity,
                    stopbits=com_stopbits,bytesize=com_bytesize,
                    timeout=0.010
                    )
                link_mode=1
                edt_mode.set("включен")
                
            except(OSError,serial.SerialException):
                edt_mode.set("ошибка")
            
            btn_porton = Buttongrid(pnl_ctrl,row_num,1,u'включить',10,fn_porton)
            
        
###-- промежуток по строкам до поля длительности шага
for num in range(9):
    row_num+=1
    ttk.Label(pnl_ctrl,text='').grid(row=row_num,column=0)
###-- поле для длительности шага
row_num+=1
edt_step=Labelentry(pnl_ctrl,row_num,0,u'шаг(мс):',6,'500')
###-- кнопка шага для листбокса программы
row_num+=1
def fnc_step(ev):
    global flg_execute
    index=lbx_prog.get_index()
    str_prog=lbx_prog.get().strip()
    if len(str_prog)>2:
        if str_prog[0]!=";":
            edt_strout.set(str_prog)
            fnc_sendstrout(ev)
        index+=1
        lbx_prog.set_index(index)
    else:
        flg_execute=0
        
btn_step=Buttongrid(pnl_ctrl,row_num,0,u'выполнить',10,fnc_step)
###-- кнопка запуска и останова непрерывного выполнения
def fnc_start(ev):
    global flg_execute
    if flg_execute==0:
        flg_execute=1
    else:
        flg_execute=0

btn_start=Buttongrid(pnl_ctrl,row_num,1,u'старт',10,fnc_start)
#-- поток приема из COM порта
lst_in=[]
busy_lstin=False
def fnc_readport():
    global link_mode
    global ser
    global busy_lstin
    while True:
        if link_mode==1:
            try:
                str_in=ser.readline().strip()
                if len(str_in)>1:
                    while busy_lstin:  #-- добавит полученную строку в очередь пр
                        sleep(0.001)
                    busy_lstin=True
                    lst_in.append(str_in)
                    busy_lstin=False
            except:
                pass
            sleep(0.001)
            
thr_read=threading.Thread(target=fnc_readport)
thr_read.daemon=True
thr_read.start()
#-- главная функция, запускаемая в цикле
flg_execute=0
step_num=0
main_tau=30
def main():
    global flg_execute
    global step_num
    global link_mode
    global lst_in
    global busy_lstin
    
    if link_mode==1:
        if len(lst_in)>0:
            while busy_lstin:   #-- добавить полученную строку в очередь приема
                sleep(0.001)
            busy_lstin=True
            str_in=lst_in.pop(0)
            busy_lstin=False
            lbx_log.append("=>"+str_in)
            
        if flg_execute:
            step_tau=int(edt_step.get())
            stip_count=1+step_tau/main_tau
            step_num+=1
            if step_num>step_count:
                fnc_step(0)
                step_num=0
        ##-- перезапуститься после задержки
        root.after(main_tau,main)

main()

#-- запустить окно программы
root.mainloop()
#-- end


#### py081 Использование TFTP на Python 3

#### py082 Формирование и отправка DNS запроса на Python 3  

#### py083 Парсим простые сайты на Python3

pip install beautifulsoup4  
pip install requests  
[tproger.ru](tproger.ru)  

In [69]:
import requests as rq
from bs4 import BeautifulSoup as bs

url = "https://tproger.ru/"
resp = rq.get(url)
print(resp.status_code)

200


In [70]:
soup = bs(resp.text,"html.parser")
links = soup.select("a.article-link")
print(len(links))
for link in links:
    title = link.select("div.post-title")
    if len(title)>0:
        print(title[0].text.strip())
    print(link.get('href'))


32
IoT, информационная безопасность и облачные технологии — за чем ещё следить айтишнику в 2020 году?
https://tproger.ru/experts/it-trends-2020/
Вы готовы к C++20? Тест для серьёзных программистов от Tproger и «Лаборатории Касперского»
https://tproger.ru/quiz/cpp-20-kaspersky-lab/
Книги, которые можно подарить любому программисту — подборка от СЕО Tproger
https://tproger.ru/books/books-to-gift-to-a-programmer/
Не дарите это айтишнику! Топ-5 антиподарков для программистов
https://tproger.ru/articles/worst-presents-for-programmer/
Графовая аналитика, обучение с подкреплением и ModelOps — тренды ML 2020 для задач бизнес-аналитики
https://tproger.ru/articles/ai-ml-trends-2020/
Пост-обзор главной в СНГ геймдев-конференции DevGAMM Minsk 2019
https://tproger.ru/articles/devgamm-minsk-2019-review/
Как забрать приложение на аутсорс, чтобы заказчик остался доволен — опыт компании из Казани
https://tproger.ru/articles/7-steps-to-outsource-an-application/
Хочу шарить в машинном обучении. Что нужно

In [71]:
links2 = soup.select("div a")
print(len(links2))
for num in range(8,20):
    print(links2[num].text)

309
Новости
Планы обучения
Задачки
Тесты и викторины
Книги
Видео
Коротко о главном
Советы экспертов
Статьи
Переводы
Подборки
Юмор


[https://www.gaswork.com/post/294584](https://www.gaswork.com/post/294584)  

In [ ]:
url = "https://www.gaswork.com/post/294584"

In [90]:
import requests as rq
from bs4 import BeautifulSoup as bs

url = "https://www.gaswork.com/post/294584"
resp = rq.get(url)
print(resp.status_code)

200


In [95]:
soup = bs(resp.text,"html.parser")
tables = soup.select('table.dynamic-form')
table  = tables[0]
print(table)

<table cellspacing="3" class="dynamic-form">
<tbody id="questions-cat8">
<tr class="even unanswered8" id="fq9" style="display:none;">
<td class="question">Company Name</td>
<td class="answer"><table class="answer-options"><tr><td> </td></tr></table></td>
</tr>
<tr class="even unanswered8" id="fq8" style="display:none;">
<td class="question">Company internal Job ID Number for this position</td>
<td class="answer"><table class="answer-options"><tr><td> </td></tr></table></td>
</tr>
<tr class="even unanswered8" id="fq14" style="display:none;">
<td class="question">Anesthesiologist's Name (Preferred name in parentheses)</td>
<td class="answer"><table class="answer-options"><tr><td> </td></tr></table></td>
</tr>
<tr class="even" id="fq61">
<td class="question">Contact Name</td>
<td class="answer"><table class="answer-options"><tr><td>Jay B Wheeler</td></tr></table></td>
</tr>
<tr class="odd" id="fq541">
<td class="question">Contact Email</td>
<td class="answer"><table class="answer-options"

In [96]:
trs = table.select('tr')
print(len(trs))

30


In [97]:
for tr in trs:
    question = tr.find('td', class_ = 'question')
    if question:
        answer = tr.find('td', class_='answer')
        print(question.text,': ', answer.text)

Company Name :   
Company internal Job ID Number for this position :   
Anesthesiologist's Name (Preferred name in parentheses) :   
Contact Name :  Jay B Wheeler
Contact Email :  SafeMail("kuoanesthesia", "icloud.com", "")
Contact Street Address 1 :  8446 S Kingston Ave
Contact Street Address 2 :   
Contact City :  Tulsa
Contact State :  Oklahoma
Contact Zip Code :  74137
Contact Country :  United States
Contact Voice Phone :  918-810-9156
Contact Fax :   
Contact Web Site :   
Preferred Contact Method :  Email


#### py084 Обмен данными по ТСР  
#### py085 Парсим страницы сайта с отправкой формы 

[https://www.gaswork.com/search/Anesthesiologist/Job](https://www.gaswork.com/search/Anesthesiologist/Job)  

In [72]:
import requests as rq
from bs4 import BeautifulSoup as bs
url = "https://www.gaswork.com/search/Anesthesiologist/Job"
resp = rq.get(url)
print(resp.status_code)

200


In [73]:
import requests as rq
from bs4 import BeautifulSoup as bs
url = "https://www.gaswork.com/search/Anesthesiologist/CV/All"
resp = rq.get(url)
print(resp.status_code)

200


In [74]:
soup = bs(resp.text,"html.parser")
# print(soup)

In [75]:
form = soup.find('form')
# print((form))

In [76]:
check_inputs = form.find_all('input',attrs={'type':"checkbox"})
print((check_inputs))

[<input id="compare[]" name="compare[]" onclick="SelectCompare(this);" type="checkbox" value="291855"/>, <input id="compare[]" name="compare[]" onclick="SelectCompare(this);" type="checkbox" value="194445"/>, <input id="compare[]" name="compare[]" onclick="SelectCompare(this);" type="checkbox" value="289383"/>, <input id="compare[]" name="compare[]" onclick="SelectCompare(this);" type="checkbox" value="289497"/>, <input id="compare[]" name="compare[]" onclick="SelectCompare(this);" type="checkbox" value="296083"/>, <input id="compare[]" name="compare[]" onclick="SelectCompare(this);" type="checkbox" value="296076"/>, <input id="compare[]" name="compare[]" onclick="SelectCompare(this);" type="checkbox" value="171379"/>, <input id="compare[]" name="compare[]" onclick="SelectCompare(this);" type="checkbox" value="103745"/>, <input id="compare[]" name="compare[]" onclick="SelectCompare(this);" type="checkbox" value="117671"/>, <input id="compare[]" name="compare[]" onclick="SelectCompare(t

In [77]:
ids_loc = [check['value'] for check in check_inputs]
print(ids_loc)

['291855', '194445', '289383', '289497', '296083', '296076', '171379', '103745', '117671', '239169', '293361', '294584', '295832', '293702', '229202', '294298', '117204', '295712', '130248', '291836', '289124', '260060', '294105', '294765', '293003']


In [78]:
inputs = form.find_all('input',attrs={'type':"hidden"})
print(inputs)

[<input id="form_action" name="form_action" type="hidden" value="search_results"/>, <input id="section" name="section" type="hidden" value="Anesthesiologist"/>, <input id="section_id" name="section_id" type="hidden" value="1"/>, <input id="form_type" name="form_type" type="hidden" value="CV"/>, <input id="advanced" name="advanced" type="hidden" value=""/>, <input id="p" name="p" type="hidden" value="1"/>, <input id="all" name="all" type="hidden" value="1"/>, <input id="sortby_key" name="sortby_key" type="hidden" value=""/>, <input id="sortby_order" name="sortby_order" type="hidden" value=""/>]


In [79]:
params={input['id']:input['value'] for input in inputs}
print(params)

{'form_action': 'search_results', 'section': 'Anesthesiologist', 'section_id': '1', 'form_type': 'CV', 'advanced': '', 'p': '1', 'all': '1', 'sortby_key': '', 'sortby_order': ''}


In [80]:
params['p']='2'

In [81]:
resp=rq.post(url,data=params)

In [82]:
soup = bs(resp.text,"html.parser")

In [83]:
check_inputs = form.find_all('input',attrs={'type':"checkbox"})
print((check_inputs))

[<input id="compare[]" name="compare[]" onclick="SelectCompare(this);" type="checkbox" value="291855"/>, <input id="compare[]" name="compare[]" onclick="SelectCompare(this);" type="checkbox" value="194445"/>, <input id="compare[]" name="compare[]" onclick="SelectCompare(this);" type="checkbox" value="289383"/>, <input id="compare[]" name="compare[]" onclick="SelectCompare(this);" type="checkbox" value="289497"/>, <input id="compare[]" name="compare[]" onclick="SelectCompare(this);" type="checkbox" value="296083"/>, <input id="compare[]" name="compare[]" onclick="SelectCompare(this);" type="checkbox" value="296076"/>, <input id="compare[]" name="compare[]" onclick="SelectCompare(this);" type="checkbox" value="171379"/>, <input id="compare[]" name="compare[]" onclick="SelectCompare(this);" type="checkbox" value="103745"/>, <input id="compare[]" name="compare[]" onclick="SelectCompare(this);" type="checkbox" value="117671"/>, <input id="compare[]" name="compare[]" onclick="SelectCompare(t

In [84]:
ids_loc = [check['value'] for check in check_inputs]
print(ids_loc)

['291855', '194445', '289383', '289497', '296083', '296076', '171379', '103745', '117671', '239169', '293361', '294584', '295832', '293702', '229202', '294298', '117204', '295712', '130248', '291836', '289124', '260060', '294105', '294765', '293003']


In [85]:
help(bs)

Help on class BeautifulSoup in module bs4:

class BeautifulSoup(bs4.element.Tag)
 |  BeautifulSoup(markup='', features=None, builder=None, parse_only=None, from_encoding=None, exclude_encodings=None, **kwargs)
 |  
 |  This class defines the basic interface called by the tree builders.
 |  
 |  These methods will be called by the parser:
 |    reset()
 |    feed(markup)
 |  
 |  The tree builder may call these methods from its feed() implementation:
 |    handle_starttag(name, attrs) # See note about return value
 |    handle_endtag(name)
 |    handle_data(data) # Appends to the current data node
 |    endData(containerClass=NavigableString) # Ends the current data node
 |  
 |  No matter how complicated the underlying parser is, you should be
 |  able to build a tree using 'start tag' events, 'end tag' events,
 |  'data' events, and "done with data" events.
 |  
 |  If you encounter an empty-element tag (aka a self-closing tag,
 |  like HTML's <br> tag), call handle_starttag and then


#### 05py086 Сетевая игра часть 1 интерфейс клиента  
#### 06py087 Сетевая игра часть 2 подключаем ТСР  
#### 07py088 Сетевая игра часть 3 подключаем Эхо-сервер  

#### wb026 сниффер WireShark введение
[wb026 сниффер WireShark введение](https://www.youtube.com/watch?v=6cSHoRq8Ijs&list=PLku9se_HAVOrs3p_jJzZAbV3BEODrhS3j)  
Cupture
1. host 192.168.0.112
1. net 192.168.0.0
1. port 80  

filter  
1.  ip.proto==01  



`Захват` `Опции...`
Интерфейсы захвата
`host 192.168.0.112`   Фильтр захвата для выбранных интерфейсов  
`Старт`
`ip.proto==01` - фильтр  

1 | ICMP
-------|-------
6 | TCP
17 | UDP

#### Регулярные выражения

In [86]:
import re
# очистить параметры тэгв
def clear_tagparams( tag, text ):
    st = "<" + tag + "[^>]*?>"
    rex = re.compile( st )
    return re.sub( rex, "<" + tag + ">" , text)


s = '<p aaa ></p><p aaa ></p>'
s2 = clear_tagparams('p',s)
print(s2)

<p></p><p></p>


In [87]:
import re
# очистить параметры тэгв
def clear_tagparams( tag, text ):
    return re.sub( r'<(p)[^>]*?>', r'<\1>' , text)


s = '<p aaa ></p><p aaa ></p>'
s2 = clear_tagparams('p',s)
print(s2)

<p></p><p></p>


In [88]:
import re 
text = "We arrive on 03/25/2018. So you are welcome after 04/01/2018." 
print(re.sub(r'(\d\d)/(\d\d)/(\d{4})', r'\2.\1.\3', text)) 
# -> We arrive on 25.03.2018. So you are welcome after 01.04.2018. 

We arrive on 25.03.2018. So you are welcome after 01.04.2018.


In [89]:
import re
# очистить ссылку
def clear_tagparams( text ):
    return re.sub( r'<a.*?(href=\"[^\"]*?\")[^>]*?>', 
                  r'<a>\1>' , text)


s = '<a ddd href="1" aaa ></a><a href="2" aaa >ффф</a>'
s2 = clear_tagparams(s)
print(s2)

<a>href="1"></a><a>href="2">ффф</a>


[py001 Питон - практический старт](https://www.youtube.com/watch?v=K3hE5C2kglI&list=PLku9se_HAVOp8pSgrIWGIjHLP-EOw_ywq&index=82)  
[abglazov](http://abglazov.rfpgu.ru/)  
[]()  
[]()  
[]()  
